In [1]:
import timemesh as tm
import numpy as np
import pandas as pd

# Load your data once for verification
df = pd.read_csv("data.csv")
input_cols = [
    "C_WD50M", "C_WS50M", "C_PS", "C_T2M", "C_QV2M",
    "N_WD50M", "N_WS50M", "N_PS", "N_T2M", "N_QV2M"
]
output_cols = ["C_WS50M", "N_WS50M", "S_WS50M", "E_WS50M", "W_WS50M"]

# =================================================================
# Case 1: Without Normalization (norm=None)
# =================================================================
loader_raw = tm.DataLoader(
    T=24,
    H=6,
    input_cols=input_cols,
    output_cols=output_cols,
    norm=None 
)
X_raw, Y_raw = loader_raw.load_csv("data.csv")

# =================================================================
# Case 2: With Min-Max Normalization
# =================================================================
loader_norm = tm.DataLoader(
    T=24,
    H=6,
    input_cols=input_cols,
    output_cols=output_cols,
    norm="MM")
X_norm, Y_norm, input_params, output_params = loader_norm.load_csv("data.csv")

# =================================================================
# Denormalize the normalized data
# =================================================================
X_denorm = tm.Normalizer.denormalize(
    X_norm,
    params=input_params,
    method="MM",
    feature_order=input_cols  # Must match original order
)

Y_denorm = tm.Normalizer.denormalize(
    Y_norm,
    params=output_params,
    method="MM",
    feature_order=output_cols
)

# =================================================================
# Verification Checks
# =================================================================
def verify_results():
    # Check 1: Raw vs Denormalized should match exactly
    x_match = np.allclose(X_raw, X_denorm, atol=1e-4)
    y_match = np.allclose(Y_raw, Y_denorm, atol=1e-4)
    
    print(f"X Match (Raw vs Denorm): {x_match}")
    print(f"Y Match (Raw vs Denorm): {y_match}")
    
    # Check 2: Normalized vs Raw ranges
    print("\nNormalization Ranges:")
    print(f"X_norm range: [{X_norm.min():.2f}, {X_norm.max():.2f}]")
    print(f"Y_norm range: [{Y_norm.min():.2f}, {Y_norm.max():.2f}]")
    
    # Check 3: Sample value comparison
    sample_idx = 0  # First sample
    time_idx = 0    # First timestep
    feature_idx = 1 # C_WS50M
    
    print("\nSample Value Comparison:")
    print(f"Original (Raw): {X_raw[sample_idx, time_idx, feature_idx]:.2f}")
    print(f"Denormalized:    {X_denorm[sample_idx, time_idx, feature_idx]:.2f}")
    print(f"Normalized:      {X_norm[sample_idx, time_idx, feature_idx]:.2f}")

verify_results()

# =================================================================
# Case 3: Test with norm=None (No normalization)
# =================================================================
def test_no_normalization():
    loader = tm.DataLoader(
        T=24,
        H=6,
        input_cols=input_cols,
        output_cols=output_cols,
        norm=None
    )
    X, Y = loader.load_csv("data.csv")
    
    # Directly compare with raw data from CSV
    expected_X = df[input_cols].values[:24]  # First window
    assert np.allclose(X[0], expected_X), "No normalization should return raw data"
    
test_no_normalization()

X Match (Raw vs Denorm): True
Y Match (Raw vs Denorm): True

Normalization Ranges:
X_norm range: [0.00, 1.00]
Y_norm range: [0.00, 1.00]

Sample Value Comparison:
Original (Raw): 8.94
Denormalized:    8.94
Normalized:      0.34


In [2]:
import timemesh as tm
import numpy as np
import pandas as pd

# Load your data once for verification
df = pd.read_csv("data.csv")
input_cols = [
    "C_WD50M", "C_WS50M", "C_PS", "C_T2M", "C_QV2M",
    "N_WD50M", "N_WS50M", "N_PS", "N_T2M", "N_QV2M"
]
output_cols = ["C_WS50M", "N_WS50M", "S_WS50M", "E_WS50M", "W_WS50M"]

# =================================================================
# Case: With Z-Score Normalization
# =================================================================
loader_z = tm.DataLoader(
    T=24,
    H=6,
    input_cols=input_cols,
    output_cols=output_cols,
    norm="Z"  # Z-score normalization
)
X_norm_z, Y_norm_z, input_params_z, output_params_z = loader_z.load_csv("data.csv")

# =================================================================
# Denormalize the Z-normalized data
# =================================================================
X_denorm_z = tm.Normalizer.denormalize(
    X_norm_z,
    params=input_params_z,
    method="Z",
    feature_order=input_cols
)

Y_denorm_z = tm.Normalizer.denormalize(
    Y_norm_z,
    params=output_params_z,
    method="Z",
    feature_order=output_cols
)

# =================================================================
# Z-Score Specific Verification
# =================================================================
def verify_zscore_results():
    # 1. Check reconstruction accuracy
    x_match = np.allclose(X_raw, X_denorm_z, atol=1e-4)
    y_match = np.allclose(Y_raw, Y_denorm_z, atol=1e-4)
    
    print(f"X Match (Raw vs Denorm-Z): {x_match}")
    print(f"Y Match (Raw vs Denorm-Z): {y_match}")
    
    # 2. Check Z-score properties
    X_flat_z = X_norm_z.reshape(-1, len(input_cols))
    print("\nZ-Score Statistics (Input Features):")
    for i, col in enumerate(input_cols):
        print(f"{col}:")
        print(f"  Mean ≈ {X_flat_z[:, i].mean():.2f} (should be ~0)")
        print(f"  Std  ≈ {X_flat_z[:, i].std():.2f} (should be ~1)")
    
    # 3. Sample value comparison
    sample_idx = 0
    time_idx = 0
    feature_idx = 1  # C_WS50M
    
    original_value = X_raw[sample_idx, time_idx, feature_idx]
    normalized_value = X_norm_z[sample_idx, time_idx, feature_idx]
    params = input_params_z[input_cols[feature_idx]]
    
    print("\nSample Value Breakdown (C_WS50M):")
    print(f"Original value: {original_value:.2f}")
    print(f"Normalized: ({original_value:.2f} - {params['mean']:.2f}) / {params['std']:.2f} = {normalized_value:.2f}")
    print(f"Denormalized: ({normalized_value:.2f} * {params['std']:.2f}) + {params['mean']:.2f} = {X_denorm_z[sample_idx, time_idx, feature_idx]:.2f}")

verify_zscore_results()

X Match (Raw vs Denorm-Z): True
Y Match (Raw vs Denorm-Z): True

Z-Score Statistics (Input Features):
C_WD50M:
  Mean ≈ -0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
C_WS50M:
  Mean ≈ 0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
C_PS:
  Mean ≈ 0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
C_T2M:
  Mean ≈ -0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
C_QV2M:
  Mean ≈ 0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
N_WD50M:
  Mean ≈ -0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
N_WS50M:
  Mean ≈ 0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
N_PS:
  Mean ≈ -0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
N_T2M:
  Mean ≈ 0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)
N_QV2M:
  Mean ≈ -0.00 (should be ~0)
  Std  ≈ 1.00 (should be ~1)

Sample Value Breakdown (C_WS50M):
Original value: 8.94
Normalized: (8.94 - 7.58) / 3.20 = 0.42
Denormalized: (0.42 * 3.20) + 7.58 = 8.94


In [3]:
import timemesh as tm
import numpy as np
import pandas as pd

# Load your data once for verification
df = pd.read_csv("data.csv")

# =================================================================
# Case 1: Without Normalization (norm=None) - Default Columns
# =================================================================
loader_raw = tm.DataLoader(
    T=24,
    H=6,
    norm=None 
)
X_raw, Y_raw = loader_raw.load_csv("data.csv")

# =================================================================
# Case 2: With Min-Max Normalization - Default Columns
# =================================================================
loader_norm = tm.DataLoader(
    T=24,
    H=6,
    norm="MM"
)
X_norm, Y_norm, input_params, output_params = loader_norm.load_csv("data.csv")

# =================================================================
# Denormalize the normalized data using ACTUAL COLUMNS FROM LOADER
# =================================================================
X_denorm = tm.Normalizer.denormalize(
    X_norm,
    params=input_params,
    method="MM",
    feature_order=loader_norm.input_cols  # Use loader's actual columns
)

Y_denorm = tm.Normalizer.denormalize(
    Y_norm,
    params=output_params,
    method="MM",
    feature_order=loader_norm.output_cols
)

# =================================================================
# Verification Checks
# =================================================================
def verify_results():
    # Check 1: Raw vs Denormalized should match exactly
    x_match = np.allclose(X_raw, X_denorm, atol=1e-4)
    y_match = np.allclose(Y_raw, Y_denorm, atol=1e-4)
    
    print(f"X Match (Raw vs Denorm): {x_match}")
    print(f"Y Match (Raw vs Denorm): {y_match}")
    
    # Check 2: Normalized vs Raw ranges
    print("\nNormalization Ranges:")
    print(f"X_norm range: [{X_norm.min():.2f}, {X_norm.max():.2f}]")
    print(f"Y_norm range: [{Y_norm.min():.2f}, {Y_norm.max():.2f}]")
    
    # Check 3: Sample value comparison using loader's columns
    sample_idx = 0  # First sample
    time_idx = 0    # First timestep
    feature_name = loader_norm.input_cols[1]  # Second feature in actual columns
    
    print("\nSample Value Comparison:")
    print(f"Feature: {feature_name}")
    print(f"Original (Raw): {X_raw[sample_idx, time_idx, 1]:.2f}")
    print(f"Denormalized:    {X_denorm[sample_idx, time_idx, 1]:.2f}")
    print(f"Normalized:      {X_norm[sample_idx, time_idx, 1]:.2f}")

verify_results()

# =================================================================
# Case 3: Test with norm=None (No normalization) - Default Columns
# =================================================================
def test_no_normalization():
    loader = tm.DataLoader(
        T=24,
        H=6,
        norm=None
    )
    X, Y = loader.load_csv("data.csv")
    
    # Get ACTUAL columns used by the loader
    actual_cols = loader.input_cols
    
    # Directly compare with raw data from CSV using THE SAME COLUMNS
    expected_X = df[actual_cols].values[:24]  # First window
    assert np.allclose(X[0], expected_X), "No normalization should return raw data"
    
test_no_normalization()

X Match (Raw vs Denorm): True
Y Match (Raw vs Denorm): True

Normalization Ranges:
X_norm range: [0.00, 1.00]
Y_norm range: [0.00, 1.00]

Sample Value Comparison:
Feature: C_WS50M
Original (Raw): 8.94
Denormalized:    8.94
Normalized:      0.34
